In [4]:
import pandas as pd

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

apipath = r'H:\\My Drive\\config\\hbqa.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)
secret_key = config['global']['OPENAI_API_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']

In [50]:
df_qa= pd.read_csv(datapath+'05-QA_Final_Auto.csv')
df_qa = df_qa.loc[df_qa['QA'].notna(),:]
df_qa.sample(4)

,Unnamed: 0,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Prompt,QA
209,323,Book03_079,324,4174,713,948,"Write 3 unique questions, corresponding answer...",\nQ1: What form did Lord Vishnu take to deceiv...
230,344,Book03_104,345,3672,645,858,"Write 2 unique questions, corresponding answer...",\nQ1: What were the sacrifices performed by So...
138,152,Book01_185,153,3977,673,895,"Write 2 unique questions, corresponding answer...",Q: What did Dhritarashtra do when he heard the...
253,417,Book03_189,418,5946,1053,1400,"Write 4 unique questions, corresponding answer...",\nQ1: What was the name of Bhrigu's wife?\nA1:...


In [51]:
df_qa.columns

Index(['Unnamed: 0', 'Section_Id', 'Chunk_Id', 'Chunk_Letters', 'Chunk_Words',
       'Chunk_Approx_Tokens', 'Prompt', 'QA'],
      dtype='object')

In [52]:
df_hbqa_auto = pd.DataFrame(columns=['Ques_Id','Section_Id', 'Chunk_Id', 'Question','Ref_Answer'])
df_hbqa_auto

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer


In [54]:
rowid=0
for row in range(df_qa.shape[0]): # For dataset
    text = df_qa.loc[row,'QA'].replace('Questions:','')
    qas = text.split('\n\n')
    # print((qas))
    for qa in qas: # For Each row, it has 5 question
        text1=qa.split('\n')
        # print(text1)
        if len(text1)==2:
            ques = text1[0][3:].strip()
            ans  = text1[1][3:].strip()
        elif len(text1)==3:
            ques = text1[1][3:].strip()
            ans  = text1[2][3:].strip()
        else:
            ques=""
            ans=""
        if ques!="":
            qa_row = rowid, df_qa.loc[row,'Section_Id'], df_qa.loc[row,'Chunk_Id'], ques, ans 
            print(qa_row)
            
            df_hbqa_auto.loc[rowid] = qa_row
            rowid+=1
              


(0, 'Book01_002', 1, 'Who asked Sauti about his journey?', 'One of the Rishis beginning the conversation asked him about his journey.')
(1, 'Book01_002', 1, 'What did the Rishis wish to hear from Sauti?', 'The Rishis wished to hear the wonderful narrations from Sauti.')
(2, 'Book01_002', 1, 'What did the Rishis ask Sauti to recite?', 'The Rishis asked Sauti to recite the sacred stories collected in the Puranas containing precepts of religious duty and of worldly profit, or the acts of illustrious saints and sovereigns of mankind.')
(3, 'Book01_002', 1, 'What is the Mahabharata?', 'The Mahabharata is a sacred work composed in elegant language, which includeth the subjects of other books, elucidated by other Shastras, and comprehendeth the sense of the four Vedas.')
(4, 'Book01_002', 1, 'What is the mystery which is threefold?', 'The mystery which is threefold is the Vedas, Yoga, and Vijnana Dharma, Artha, and Kama.')
(5, 'Book01_002', 2, 'Who asked Sauti about his journey?', 'One of the

In [55]:
df_hbqa_auto

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer
0,0,Book01_002,1,Who asked Sauti about his journey?,One of the Rishis beginning the conversation a...
1,1,Book01_002,1,What did the Rishis wish to hear from Sauti?,The Rishis wished to hear the wonderful narrat...
2,2,Book01_002,1,What did the Rishis ask Sauti to recite?,The Rishis asked Sauti to recite the sacred st...
3,3,Book01_002,1,What is the Mahabharata?,The Mahabharata is a sacred work composed in e...
4,4,Book01_002,1,What is the mystery which is threefold?,"The mystery which is threefold is the Vedas, Y..."
...,...,...,...,...,...
1080,1080,Book03_224,450,What was the name of the king who conquered th...,"The king's name was Uparichara, also known as ..."
1081,1081,Book03_224,450,What did Indra give to King Vasu for his grati...,"Indra gave King Vasu a crystal car, a triumpha..."
1082,1082,Book03_224,450,How many sons did King Vasu have?,King Vasu had five sons of great energy and im...
1083,1083,Book03_224,450,What did Indra tell King Vasu about those who ...,Indra told King Vasu that those who joyously o...


In [56]:
df_hbqa_auto['WordsInQues'] = df_hbqa_auto['Question'].str.split(' ').apply(len)
df_hbqa_auto['WordsInAns']  = df_hbqa_auto['Ref_Answer'].str.split(' ').apply(len)
df_hbqa_auto.head(5)

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer,WordsInQues,WordsInAns
0,0,Book01_002,1,Who asked Sauti about his journey?,One of the Rishis beginning the conversation a...,6,12
1,1,Book01_002,1,What did the Rishis wish to hear from Sauti?,The Rishis wished to hear the wonderful narrat...,9,10
2,2,Book01_002,1,What did the Rishis ask Sauti to recite?,The Rishis asked Sauti to recite the sacred st...,8,32
3,3,Book01_002,1,What is the Mahabharata?,The Mahabharata is a sacred work composed in e...,4,29
4,4,Book01_002,1,What is the mystery which is threefold?,"The mystery which is threefold is the Vedas, Y...",7,15


In [ ]:
df_hbqa_auto.to_csv(datapath + '06.1-HBQA_Auto.csv', index=None)
print(df_hbqa_auto.shape)
df_hbqa_auto.head(5)